In [4]:
import pandas as pd
import numpy as np
import os
os.environ['MKL_NUM_THREADS'] = '1'
from collections import Counter
import itertools
import json
import scipy.sparse as sparse
import pickle
import torch
import misc.util as util
from torch import nn

import importlib
from torch.utils.data import DataLoader
from misc.loader import AEDataset
from misc.util import *
from tqdm.auto import tqdm
from eval.rec_eval import *
import neuralsort.pl as pl
from models.loss import neuPrecLoss
from misc.loader import RecDataset
import models
from models.loss import *

In [5]:
import pickle
with open("data/parsed/ml-1m-new", 'rb') as f:
    (tr_users, val_users, te_users, train_data, val_tr, val_te, te_tr, te_te) = pickle.load(f)
empty = csr_matrix(train_data.shape)

In [6]:
n_users, n_items = train_data.shape
ae_dataset = AEDataset(train_data[tr_users])

In [7]:
vin = val_tr[val_users]
vo = val_te[val_users]
vin = val_tr[val_users]
vo = val_te[val_users]
batch_size=256

In [8]:
qed = []
import models.ae
importlib.reload(models.ae)
import neuralsort.neuralobjs
importlib.reload(models.ae)

use_vae=False
loader = DataLoader(ae_dataset, batch_size=batch_size, shuffle=True, num_workers=6)
if use_vae:
    model = models.ae.MultiVAE([64] + [n_items], dropout=0.5)
else:
    model = models.ae.MultiDAE([64] + [n_items], dropout=0.5)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
sc = neuralsort.neuralobjs.SC()
lm = -1 

update_count = 0
total_anneal_steps = 30000
anneal_cap = 0.3
for epoch in (range(1, 50 + 1)):
    model = model.train()
    model.training = True
    tr_losses = []
    # train for one epoch
    for uid, rowl in (loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        loss = None 
        if use_vae:
            scores, mu, logvar = model.forward(row)
            if total_anneal_steps > 0:
                anneal_cap = min(anneal_cap, 1. * update_count / total_anneal_steps)
            else:
                anneal = anneal_cap
            loss  = models.ae.loss_function(scores, row, mu, logvar)
            update_count += 1
        else:
            scores = model.forward(row)
            loss  = models.loss.MSELoss(row, scores)
        (loss.mean()).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        tr_losses.append(loss.detach().unsqueeze(-1)) 
        optimizer.step()
    
    if (epoch % 1 == 0):
        vind = torch.FloatTensor(np.asarray(vin.todense()).astype(np.float32)).cuda()
        if use_vae is True:
            scores, mu, logvar = model.forward(vind)
            val_loss = models.ae.loss_function(scores, vind, mu, logvar).mean()
        else:
            scores = model.forward(vind)
            val_loss = models.loss.MultinomialLoss(vind, scores).mean().detach().cpu().numpy()
        tr_loss = torch.cat(tr_losses).mean()

        model.eval()
        model.training=False
        wrapper1 = models.ae.implicitWrapper(model, train_data, naive_sparse2tensor, vae=True)
        wrapper2 = models.ae.implicitWrapper(model, vin, naive_sparse2tensor, vae=True)
        tr_ndcg = (ranking_metrics_at_k(wrapper1, empty, train_data, K=10, num_threads=4)['map'])
        val_ndcg = (ranking_metrics_at_k(wrapper2, vin, vo, K=10, num_threads=4)['map'])

        ret = (tr_loss.detach().cpu().numpy(), val_loss, tr_ndcg, val_ndcg)
        print(["%0.4f" % x for x in ret])
        qed.append(ret)
no_add = qed

OSError: [Errno 12] Cannot allocate memory

In [2]:
def sample_gumbel(samples_shape, eps = 1e-10):
    U = torch.rand(samples_shape).cuda()
    return -torch.log(-torch.log(U + eps) + eps)
def deterministic_NeuralSort(s, tau=0.1):
    """
    s: input elements to be sorted. Shape: batch_size x n x 1
    tau: temperature for relaxation. Scalar.
    """
    n = s.size()[1]
    one = torch.ones((n, 1), dtype = torch.float32)
    A_s = torch.abs(s - s.permute(0, 2, 1))
    B = torch.matmul(A_s, torch.matmul(one, torch.transpose(one, 0, 1)))
    scaling = (n + 1 - 2 * (torch.arange(n) + 1)).type(torch.float32)
    C = torch.matmul(s, scaling.unsqueeze(0))
    P_max = (C-B).permute(0, 2, 1)
    sm = torch.nn.Softmax(-1)
    P_hat = sm(P_max / tau)
    return P_hat

In [3]:
def neuPrecLoss(sc, scores, row, topk=100, k=5, tau=10.0, use_top=True):
    scores = scores + sample_gumbel(scores.shape).cuda()
    if use_top:
        y_hat, indices = torch.topk(scores, topk)
        y = row.gather(1, indices)
    else:
        indices = torch.randint(0, row.shape[1], size=(row.shape[0], topk)).cuda()
        y_hat = scores.gather(1, indices)
        y = row.gather(1, indices)
    ysum = y.sum(1)
    #y_hat = y_hat + 
    yy = sc(y_hat, k=k)
    loss = -((y * (yy.sum(1))).sum(1) * (1 / (1e-10 + ysum)) * torch.clamp(ysum, 0, 1))
    return loss

In [9]:
qed = []
import models.ae
importlib.reload(models.ae)
import neuralsort.neuralobjs
importlib.reload(models.ae) 
use_vae=False
loader = DataLoader(ae_dataset, batch_size=batch_size, shuffle=True, num_workers=6)
if use_vae:
    model = models.ae.MultiVAE([128] + [n_items], dropout=0.5)
else:
    model = models.ae.MultiDAE([128] + [n_items], dropout=0.5)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
sc = neuralsort.neuralobjs.SC()
lm = -1 
batch_size=64
update_count = 0
total_anneal_steps = 30000
anneal_cap = 0.3
for epoch in (range(1, 50 + 1)):
    model = model.train()
    model.training = True
    tr_losses = []
    # train for one epoch
    for uid, rowl in tqdm(loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        loss = None 
        if use_vae:
            scores, mu, logvar = model.forward(row)
            if total_anneal_steps > 0:
                anneal_cap = min(anneal_cap, 1. * update_count / total_anneal_steps)
            else:
                anneal = anneal_cap
            loss  = models.ae.loss_function(scores, row, mu, logvar)
            update_count += 1

        else:
            scores = model.forward(row)
            loss  = models.loss.MultinomialLoss(row, scores)
        loss2 = neuPrecLoss(sc, scores, row, topk=300, k=10, tau=1.0, use_top=True)
        (1.0 * loss.mean() + 10.0 * loss2.mean()).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        tr_losses.append(loss.detach().unsqueeze(-1)) 
        optimizer.step()
    
    if (epoch % 10 == 0):
        #vind = torch.FloatTensor(np.asarray(vin.todense()).astype(np.float32)).cuda()
        #if use_vae is True:
            #scores, mu, logvar = model.forward(vind)
            #val_loss = models.ae.loss_function(scores, vind, mu, logvar).mean()
        #else:
            #scores = model.forward(vind)
            #val_loss = models.loss.MultinomialLoss(vind, scores).mean().detach().cpu().numpy()
        tr_loss = torch.cat(tr_losses).mean()
        scores, val_loss = 0, 0
        model.eval()
        model.training=False
        wrapper1 = models.ae.implicitWrapper(model, train_data, naive_sparse2tensor, vae=True)
        wrapper2 = models.ae.implicitWrapper(model, vin, naive_sparse2tensor, vae=True)
        #tr_ndcg = (ranking_metrics_at_k(wrapper1, empty, train_data, K=10, num_threads=4)['map'])
        val_ndcg = (ranking_metrics_at_k(wrapper2, vin, vo, K=10, num_threads=4)['map'])

        ret = (tr_loss.detach().cpu().numpy(), val_loss, tr_ndcg, val_ndcg)
        print(["%0.4f" % x for x in ret])
        qed.append(ret)
add = qed

OSError: [Errno 12] Cannot allocate memory